In [1]:
# modified from code snippet from yair carmon, thanks yair!
import wandb
import pandas as pd
import tqdm
import pickle
from matplotlib import pyplot as plt
import os
from pathlib import Path

In [2]:
"""
Download logs
"""

# Replace 'entity' with the name of the user or team and 'project' with the name of the project
entity = 'samir'
project = 'dcnlp'

# Initialize the wandb API
api = wandb.Api()

run_cache = set([Path(p).stem for p in os.listdir("exp_data/models")])

# Fetch runs from the specified project that match the name criteria
runs = api.runs(f"{entity}/{project}", filters={"display_name": {"$regex": "^(rpj|c4_original|rw_original)-.*"}})

results = []
hit_cache = set()

# Iterate over the runs and extract the required information

runs = [run for run in runs if run.name in run_cache]

for run in tqdm.tqdm(runs):
    # Extract run name
    run_name = run.name

    # Initialize containers for the series data
    train_loss_series = []
    tokens_series = []
    
    # Access the history of each run for 'train/loss' and 'tokens'
    for row in run.scan_history(keys=["train/loss", "tokens"]):
        # Extract train/loss and tokens if available
        train_loss = row.get("train/loss")
        tokens = row.get("tokens")
        
        if train_loss is not None:
            train_loss_series.append(train_loss)
        if tokens is not None:
            tokens_series.append(tokens)

    results.append((run_name, pd.Series(train_loss_series, index=pd.Index(tokens_series, name='tokens'), name='train_loss')))

with open('scaling_train_loss.pkl', 'wb') as f:
    pickle.dump(results, f)

100%|██████████| 163/163 [02:53<00:00,  1.06s/it]


In [3]:
results = None
with open('scaling_train_loss.pkl', 'rb') as f:
    results = pickle.load(f)

tmp = []
for name, df_ in pd.DataFrame(results, columns=['name', 'loss']).groupby('name'):
    melded_loss = pd.concat(df_.loss.values).groupby('tokens').mean().astype(float)
    if len(melded_loss) > 0:
        tmp.append((name, melded_loss))
loss_df = pd.DataFrame(tmp, columns=['name', 'loss'])


/var/folders/g_/5xx_z0dx7mq4336sz8tg1gb00000gq/T/ipykernel_81966/3579488461.py:7: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  melded_loss = pd.concat(df_.loss.values).groupby('tokens').mean().astype(float)


In [4]:
# name of first run
loss_df.iloc[0, :]["name"]

'c4_original-d=1024_l=24_h=8-0.25'

In [5]:
# token loss pairs
loss_df.iloc[0, :]["loss"]

tokens
2.097152e+06    11.021726
2.306867e+07     9.303206
4.404019e+07     8.256299
6.501171e+07     7.316289
8.598323e+07     6.891155
                  ...    
1.986003e+09     3.532320
2.006974e+09     3.562413
2.027946e+09     3.540241
2.048918e+09     3.529097
2.057306e+09     3.525357
Name: train_loss, Length: 108, dtype: float64